In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

from crewai_tools import PDFSearchTool, SerperDevTool, ScrapeWebsiteTool
from crewai import Agent, Task, Crew, Process
# from crewai_tools import SerperDevTool, FileReadTool, ScrapeWebsiteTool, WebsiteSearchTool

c:\Users\yapch\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv('.env')
os.environ['OPENAI_MODEL_NAME'] = "gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
topic = input("Enter the audit topic: ")
if topic:
    print("You entered:", topic)
else:
    print("Please enter an audit task.")

You entered: baby bonus scheme


In [4]:
tool_pdf = PDFSearchTool(pdf='./Files/AR_2024.pdf')
tool_search = SerperDevTool(n_results=5, country="sg")
tool_webscrape = ScrapeWebsiteTool()

2024-10-08 18:02:43,328 - 17760 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 295 0 (offset 0)
2024-10-08 18:02:43,328 - 17760 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 669 0 (offset 0)
2024-10-08 18:02:43,329 - 17760 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 681 0 (offset 0)
Inserting batches in chromadb: 100%|██████████| 3/3 [00:15<00:00,  5.23s/it]


In [5]:
auditor = Agent(
    role = "auditor",
    goal="Return the Audit Planning Memo on {topic} based on inputs from researcher and past audit findings from the audit assistant.",
    backstory = """You are an auditor. You need to consider the inputs from the researcher and audit assistant to determine what to write for the audit planning memo and risk assessment""",
    allow_delegation = True,
    max_iter = 15,
    verbose = True
)

researcher = Agent(
    role="researcher",
    goal="From the internet, research and analyse and sieve out information related to {topic} only." ,
    backstory="""As a researcher, navigating and extracting critical information is crucial. You are assisting the auditor to research on topics relevant to {topic}. 
    """,
    allow_delegation=False,
    verbose=True,
    max_iter = 15
)

Audit_asssitant = Agent(
    role = "Audit assistant",
    goal ="From the file provided, extract the audit findings pertaining to {topic} only. Pass the audit findings to auditor",
    backstory = "You are an audit assistant. Your job is to extract the past audit findings relating to the {topic} and pass it to the auditor.",
    allow_delegation=False,
    verbose=True,
    max_iter = 15
)

task_search = Task(
    description="""Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only. Ignore urls that ends with .pdf.
    """,
    expected_output="""\
    Returns websites that contain the top 5 most relavant policy and regulation information on {topic} only. The output must be in JSON format with the description as the key and url as value.""",

    agent=researcher,
    tools=[tool_search],
)


task_websitesearch = Task(
    description="""Scrape through a website for information related to {topic} only
    """,
    expected_output="""\
    Summarise the website for information related to {topic} only in the Singapore context. """,

    agent=researcher,
    tools=[tool_webscrape],
    context = [task_search]
)

task_extract_findings = Task(
    description = """Read the pdf file and return the audit findings pertaining to the {topic} and pass it to the auditor""",
    expected_output= """A list of audit findings pertainint to the {topic}""",
    tools=[tool_pdf],
    agent = Audit_asssitant
)

task_write= Task(
    description="""\
    1. Use the content to craft an audit planning memo and risk assessment based on {topic} .
    2. Sections are properly named in an engaging manner.
    3. Proofread for grammatical errors and alignment with the common style used in audit planning memos.""",

    expected_output="""
    A well-written audit planning memo and risk assessment in markdown format. Each section should have minimum of 2 and maximum of 5 paragraphs.""",
    agent=auditor,
    context=[task_websitesearch, task_extract_findings],
    output_file="APM.md"
)




In [6]:
#creating the crew

crew = Crew(
    agents=[auditor,researcher,Audit_asssitant],
    tasks=[task_search,task_websitesearch,task_extract_findings,task_write],
    verbose=True
)


In [7]:
result = crew.kickoff(inputs={"topic": topic})

 [2024-10-08 18:04:56][DEBUG]: == Working Agent: researcher
 [2024-10-08 18:04:56][INFO]: == Starting Task: Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on baby bonus scheme only. Ignore urls that ends with .pdf.
    


> Entering new CrewAgentExecutor chain...
Action: Search the internet  
Action Input: {"search_query":"Singapore baby bonus scheme policy regulation"}   


Search results: Title: Baby Bonus Scheme - Made For Families
Link: https://www.madeforfamilies.gov.sg/support-measures/raising-your-child/financial-support/baby-bonus-scheme
Snippet: Your child must obtain Singapore Citizenship before 24 months of age to be eligible for the Cash Gift, and before 12 years of age to be eligible for the Child ...
---
Title: [PDF] terms and conditions for the baby bonus scheme
Link: https://www.babybonus.msf.gov.sg/Documents/Terms%20and%20Conditions%20for%20Baby%20Bonus%20Scheme%20

In [8]:
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")

Raw Output: ```markdown
# Audit Planning Memo for the Baby Bonus Scheme

## Introduction
The Baby Bonus Scheme in Singapore represents a significant governmental initiative aimed at supporting families with the financial burdens of raising children. As part of the broader Marriage and Parenthood Package, this program includes the Baby Bonus Cash Gift (BBCG) and the Child Development Account (CDA). This memo outlines the audit planning and risk assessment for the Baby Bonus Scheme, drawing on insights from previous audits and current operational processes.

## Objectives of the Audit
The primary objectives of this audit are to evaluate the effectiveness of the Baby Bonus Scheme in achieving its intended goals, ensure compliance with established eligibility criteria, assess the efficiency of the application and payment processes, and identify areas for improvement. By conducting a thorough review, we aim to enhance the transparency and accountability of the scheme, thereby ensuring that 

In [ ]:
# tool = SerperDevTool(n_results=3, country="sg")

# agent = Agent(
#     role="search agent",
#     goal="return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}" ,
#     backstory="""You're working on gathering all relavant policy and regualtion information on {topic}.
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter=40
# )

# task = Task(
#     description="""return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}
#     """,
#     expected_output="""\
#     A detailed report with most relevant information information that is related to policy and regulation on {topic}.""",

#     agent=agent,
#     tools=[tool],
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     verbose=True
# )

# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
# print(f"Raw Output: {result.raw}")
# print(f"Token Usage: {result.token_usage}")
# print("-----------------------------------------\n\n")
# print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
# print("-----------------------------------------\n\n")
# # print(f"Tasks Output of Task 2: {result.tasks_output[1]}")